In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [2]:
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
from datetime import datetime
import requests, os

In [3]:
settings = {
    'host': os.environ.get('ACCOUNT_HOST',  'https://afdiaz.documents.azure.com:443/'),
    'master_key': os.environ.get('ACCOUNT_KEY', 'jtPGX3mOrlB2MwwoHMmNeTlWnmuc1ji3DKBaQzDpDrTyha7Yf4XhVec8CFGOLUfWnmBRwjSNnHglACDbT7tv9g=='),
    'database_id': os.environ.get('COSMOS_DATABASE', 'base2'),
    'container_id': os.environ.get('COSMOS_CONTAINER', 'prueba2')

}

#Asignación de valores a variables internas
HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']
BASE_URL = "https://api.shodan.io/"


In [4]:
#buscan la base datos 
if __name__ == '__main__':
    client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY}, user_agent="CosmosDBPythonQuickstart", user_agent_overwrite=True)
    try:
        # setup database for this sample
        try:
            db = client.create_database(id=DATABASE_ID)
            print('Database with id \'{0}\' created'.format(DATABASE_ID))

        except exceptions.CosmosResourceExistsError:
            db = client.get_database_client(DATABASE_ID)
            print('Database with id \'{0}\' was found'.format(DATABASE_ID))

        # setup container for this sample
        try:
            container = db.create_container(id=CONTAINER_ID, partition_key=PartitionKey(path='/partitionKey'))
            print('Container with id \'{0}\' created'.format(CONTAINER_ID))

        except exceptions.CosmosResourceExistsError:
            container = db.get_container_client(CONTAINER_ID)
            print('Container with id \'{0}\' was found'.format(CONTAINER_ID))

    except exceptions.CosmosHttpResponseError as e:
        print('\nrun_sample has caught an error. {0}'.format(e.message))

    finally:
            print("\nrun_sample done")


run_sample has caught an error. (Forbidden) Subscription owning the database account is disabled.
ActivityId: 3beba8e3-88f5-43d6-a459-78cb9ce0c2a6, Microsoft.Azure.Documents.Common/2.14.0
Code: Forbidden
Message: Subscription owning the database account is disabled.
ActivityId: 3beba8e3-88f5-43d6-a459-78cb9ce0c2a6, Microsoft.Azure.Documents.Common/2.14.0

run_sample done


In [5]:
def almacenar_estadisticas_puertos(ip):
    # Obtener estadísticas de puertos para la IP específica
    headers = {'X-Key': '9f637276-ba48-457f-9a0e-1ce9786b3287'}
    response = requests.get(f"https://api.binaryedge.io/v2/query/search/stats?query=ip:{ip}&type=ports", headers=headers)
    ports_stats = response.json()
    

    if not ports_stats:
        print("No se pudieron obtener las estadísticas de puertos de BinaryEdge")
        return
          # Intentar crear el contenedor. Si ya existe, obtenerlo.
    try:
        container = db.create_container(id=ip, partition_key=PartitionKey(path='/partitionKey'))
        item = {
        'id': f"port_{ip}",
        'partitionKey': 'Test1',
        'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'portsDescription': ports_stats
        }
        container.create_item(body=item)
        print(f"Estadísticas para el puerto {item['id']} almacenadas en Cosmos DB")
        print(f'Contenedor con ID \'{id}\' creado.')
    except exceptions.CosmosResourceExistsError:
        print(f'Contenedor con ID \'{id}\' fue encontrado.')

In [6]:
almacenar_estadisticas_puertos(" 80.188.166.75")

NameError: name 'db' is not defined

In [ ]:
import requests
import socket
BASE_URL = "https://api.shodan.io/"

def get_data_about_an_ip(ip_address):
    api_key = "zd1iJ79xWHhnhKb2jlzdtjr56VXuUgA8"
    url = f"{BASE_URL}shodan/host/{ip_address}?key={api_key}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()  
        return response.json()
    except requests.exceptions.HTTPError as http_err:
        print(f'Error HTTP: {http_err}')
    except requests.exceptions.RequestException as req_err:
        print(f'Request error: {req_err}')
    except Exception as err:
        print(f'Error desconocido: {err}')
    
    return None

# Ejemplo de uso
ip_to_check = "75.58.232.10"
data = get_data_about_an_ip(ip_to_check)

if data:
    print(data)
else:
    print("No se pudieron obtener datos.")


In [ ]:
def check_ip_vulnerability(ip_address, port_to_check):
    data = get_data_about_an_ip(ip_address)
    open_ports = []
    closed_ports = []
    vulnerability_status = False

    if data and "data" in data:
        for entry in data["data"]:
            if "vulns" in entry and "CVE-2019-0708" in entry["vulns"]:
                print(f"La dirección IP {ip_address} es vulnerable.")
                vulnerability_status = True
                break
            if "ports" in entry:
                open_ports = entry["ports"]
                if port_to_check in open_ports:
                    print(f"El puerto {port_to_check} está ABIERTO en la dirección IP {ip_address}.")
                else:
                    print(f"El puerto {port_to_check} está CERRADO en la dirección IP {ip_address}.")
            else:
                print(f"No hay información de puertos para la dirección IP {ip_address}.")

        if open_ports:
            print("Lista de puertos abiertos:")
            print(open_ports)
        else:
            print("No hay información de puertos abiertos.")

        return vulnerability_status
    else:
        print(f"No se pudo obtener datos para la dirección IP {ip_address}.")
        return False

# Simulación de la función GetDataAboutAnIP
def get_data_about_an_ip(ip_address):
    # Devolver datos simulados con información de puertos abiertos y vulnerabilidades
    return {
        "data": [
            {
                "ports": [3389],  # Lista de puertos abiertos

                "vulns": ["CVE-2019-0708"]  # Lista de vulnerabilidades (puedes ajustar según tus necesidades)
            }
        ]
    }

# Ejemplo de uso
ip_to_check = "75.58.232.10"
# Cambia la tupla a un solo puerto
port_to_check = 3389

vulnerability_result = check_ip_vulnerability(ip_to_check, port_to_check)

if vulnerability_result:
    print("Tomar medidas para abordar la vulnerabilidad.")
else:
    print("La dirección IP está segura.")


In [ ]:
import socket
from datetime import datetime
from azure.cosmos import exceptions, CosmosClient
import json

def verificar_vulnerabilidad(ip, open_ports, all_ports):
    try:
        data = get_data_about_an_ip(ip)
        vulnerabilities_present = any("vulns" in entry and "CVE-2019-0708" in entry["vulns"] for entry in data.get("data", []))
        
        if open_ports and vulnerabilities_present:
            print("Vulnerable")
            return True
        else:
            print("No vulnerable")
            return False
    except Exception as e:
        print(f"Error al verificar la vulnerabilidad: {str(e)}")
        return False


def is_port_open(ip, port):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.settimeout(5)
        return s.connect_ex((ip, port)) == 0

def get_data_about_an_ip(ip_address):
    # Simulación de la lógica para obtener datos sobre la IP
    return {
        "data": [
            {"ports": [3389, 80], "vulns": ["CVE-2019-0708"]}
        ]
    }

def save_to_cosmos(ip, vulnerability_status, open_ports, all_ports):
    endpoint = "https://afdiaz.documents.azure.com:443/"
    key = "jtPGX3mOrlB2MwwoHMmNeTlWnmuc1ji3DKBaQzDpDrTyha7Yf4XhVec8CFGOLUfWnmBRwjSNnHglACDbT7tv9g=="
    database_name = "base2"
    container_name = "prueba2"

    client = CosmosClient(endpoint, key)
    database = client.get_database_client(database_name)
    container = database.get_container_client(container_name)

    date_now = datetime.now().strftime('%y-%m-%d %H:%M:%S')
    item_id = str(ip)
    
    informacion = {
        'id': item_id,
        'last_update': date_now,
        'info_vulnerabilidad': vulnerability_status,
        'puertos_abiertos': open_ports,
        'puertos_cerrados': [port for port in all_ports if port not in open_ports],
        'partitionKey': 'Account1',
    }

    try:
        item = container.read_item(item=item_id, partition_key='Account1')
        container.replace_item(item=item, body=informacion)
        print("Item actualizado")
    except exceptions.CosmosResourceNotFoundError:
        container.create_item(body=informacion)
        print("Item creado")

    item = container.read_item(item=item_id, partition_key='Account1')
    print(json.dumps(item, indent=2))

# Ejemplo de uso
ip_to_check = '75.58.232.10'
all_ports_to_check = [3389, 80, 443, 22]  # Agrega todos los puertos que deseas verificar
open_ports = [port for port in all_ports_to_check if is_port_open(ip_to_check, port)]
vulnerability_status = verificar_vulnerabilidad(ip_to_check, open_ports, all_ports_to_check)

save_to_cosmos(ip_to_check, vulnerability_status, open_ports, all_ports_to_check)

if vulnerability_status:
    print(f"La IP {ip_to_check} es vulnerable y los puertos {open_ports} están abiertos.")
else:
    print(f"La IP {ip_to_check} no es vulnerable o los puertos {open_ports} están cerrados.")
